In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from analysis.modules.moving_average import moving_average_analysis

In [6]:
df = moving_average_analysis('SSI', investment_preference = [(10,20),(20,50),(50, 100)])
df.tail()

,stock_exchange,stock_code,date,open_price,highest_price,lowest_price,close_price,volume,sma_10,ema_10,...,sma_50,ema_50,sma_100,ema_100,state_sma_10_20,state_ema_10_20,state_sma_20_50,state_ema_20_50,state_sma_50_100,state_ema_50_100
3450,HSX,SSI,2021-09-01,61.3,62.0,60.90,61.20,9400800,60.125,59.319104,...,55.343728,53.863491,43.831414,45.569699,wait,wait,wait,wait,wait,wait
3451,HSX,SSI,2021-09-06,61.4,61.7,60.00,60.00,17069400,59.855,59.442904,...,55.834656,54.104139,44.155263,45.855448,bull,wait,wait,wait,wait,wait
3452,HSX,SSI,2021-09-07,60.5,61.4,59.80,60.40,24553700,59.775,59.616921,...,56.351000,54.351035,44.451392,46.143459,wait,wait,wait,wait,wait,wait
3453,HSX,SSI,2021-09-09,44.4,45.3,43.75,43.95,20144300,57.830,56.768390,...,56.130000,53.943151,44.596082,46.100024,wait,bull,wait,wait,wait,wait
3454,HSX,SSI,2021-09-10,44.0,44.3,43.60,43.80,10409900,56.010,54.410501,...,55.914000,53.545381,44.735540,46.054479,wait,wait,wait,wait,wait,wait
